# Day 3: Learning Structured Predictors

In Day 2, we focused on generative sequence classifiers - HMMs. Today's focus is on discriminative classifiers. Recall that:

* **generative classifiers** try to model the probability distribution of the data, $P(X, Y)$;

* **discriminative classifiers** only model the conditional probability of each class given the observed data, $P(Y\,|\,X)$.

In Day 1, we implemented discriminative models for classification tasks. Today, we extend this concept to the classification of _sequential_ data.

## Summary

You will be using two discriminative classifiers to do part-of-speech tagging:
* Conditional Random Fields (CRF) and
* Structured Perceptron.

Your tasks for this lab session are:

* to train a CRF model using two different sets of features (exercises 3.1 and 3.2); 
* to implement the structured perceptron algorithm (exercise 3.3); 
* to compare the performance of the Structured Perceptron with that of CRFs (exercise 3.4).

## Introduction

Discriminative sequence models aim to solve the following:

$$\underset{y\,\in\,\Lambda^N}{\textrm{arg max}}\ P(Y=y\,|\,X=x)=\underset{y\,\in\,\Lambda^N}{\textrm{arg max}}\ \boldsymbol{w}\cdot\boldsymbol{f}(x, y)$$

where $\boldsymbol{w}$ is the model's weight vector, and $\boldsymbol{f}(x, y)$ is a feature vector. Notice that now both $y$ and $x$ are $N$-dimensional vectors, whereas in Day 1, these variables were just scalar numbers.

In Day 2, sequences were scored using the log-probability. On today's models we are still scoring the sequences; the only difference is the scores are now computed as the product of the weights with the feature vector:


| score | Hidden Markov Models (Day 2) | Discriminative Models (Today) |
| ------------------------------- | ---------------- | ---------------- |
| $\textrm{score}_\textrm{emiss}$ | $\log P(x_i\,|\,y_i) $ | $\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{emiss}(i, x, y_i)$ |
| $\textrm{score}_\textrm{init}$ | $\log P(y_1\,|\,\mathrm{start}) $ | $\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{init}(x, y_1)$ |
| $\textrm{score}_\textrm{trans}$ | $\log P(y_{i+1}\,|\,y_i) $ | $\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{trans}(i, x, y_i, y_{i+1})$ |
| $\textrm{score}_\textrm{final}$ | $\log P(\mathrm{stop}\,|\,y_N) $ | $\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{final}(x, y_N)$ |

Notice that the scores computed using the feature vector depend on two sequential values of the output variable, $y$, but may depend on the whole observated input, $x$. We can now rewrite the above expression as

$$
\underset{y\,\in\,\Lambda^N}{\textrm{arg max}}\ 
\sum_{i=1}^N \boldsymbol{w}\cdot\boldsymbol{f}_\textrm{emiss}(i, x, y_i) + 
\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{init}(x, y_1) + 
\sum_{i=1}^{N-1}\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{trans}(i, x, y_i, y_{i+1}) + 
\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{final}(x, y_N) = 
\\
\underset{y\,\in\,\Lambda^N}{\textrm{arg max}}\ 
\sum_{i=1}^N \textrm{score}_\textrm{emiss}(i, x, y_i) + 
\textrm{score}_\textrm{init}(x, y_1) + 
\sum_{i=1}^{N-1}\textrm{score}_\textrm{trans}(i, x, y_i, y_{i+1}) +
\textrm{score}_\textrm{final}(x, y_N)
$$

The advantage of these linear models over HMMs is that the feature vector may contain more general features. [TODO example of what these 'standard' and 'extended' features could be]

### Decoding

One important thing to notice is that the decoding process - the process by which we pick the most likely label $y_i$ for the observation $x_i$ - stays the same. This means *we do not need to develop new decoders,* only new functions to compute the scores. Because of this, we will keep using the Viterbi and Forward-Backward algorithms developed on Day 2.

### Training the classifier

Today we will cover two different approaches to training sequential discriminative models. Given a training set with $M$ observation-label pairs, $\{(x_m, y_m)\}_{m=1}^M$ (note that $x_m$, $y_m$ are $N$-dimensional vectors, as $m$ indexes the training sample):

* **Conditional Random Fields** maximize the log-likelihood over $w$ on the observed training set.
* **Structured Perceptron** iteratively updates $w$ in order to correctly classify the training set.

## Conditional Random Fields

CRFs are the generalization of the Maximum Entropy classifier for sequences. The general concept is the same, with a couple of diferences to be discussed below. They are trained by solving the following optimization problem:

$$
\hat{w} = \underset{w}{\textrm{arg max}}\ \sum_{m=1}^M\log P_w(Y=y_m\,|\,X=x_m)
$$

where [TODO fix parenthesis... apparently \left( doesnt work in notebooks]

$$
P_w(Y=y_m\,|\,X=x_m) =
\frac{1}{Z(w, x)}\ \exp (\sum_{i=1}^N \boldsymbol{w}\cdot\boldsymbol{f}_\textrm{emiss}(i, x, y_i) + 
\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{init}(x, y_1) + 
\sum_{i=1}^{N-1}\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{trans}(i, x, y_i, y_{i+1}) + 
\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{final}(x, y_N))\\
Z(w, x) = \sum_{y\,\in\,\Lambda^N} \exp (\sum_{i=1}^N \boldsymbol{w}\cdot\boldsymbol{f}_\textrm{emiss}(i, x, y_i) + 
\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{init}(x, y_1) + 
\sum_{i=1}^{N-1}\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{trans}(i, x, y_i, y_{i+1}) + 
\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{final}(x, y_N))
$$

As before, the partition function $Z(w,x)$ ensures the sum of probabilities over all possible labels $y\in\Lambda^N$ is equal to 1.

To avoid overfitting, it is common to add the Euclidean norm function as a regularization term. This is equivalent
to considering a zero-mean Gaussian prior on the weight vector. The optimization problem becomes:

$$
\hat{w} = \underset{w}{\textrm{arg max}}\ \sum_{m=1}^M\log P_w(Y=y_m\,|\,X=x_m) - \frac{\lambda}{2}||w||^2
$$

which is precisely the structured variant of the maximum entropy method discussed on Day 1. Unlike with HMMs, the above problem has to be solved numerically.

### Differences with respect to ME algorithm

* CRe does not compute posterior marginals, $P(Y=y\,|\,X=x)$ for every possible $y\in\Lambda^N$, as there are exponentially many possible $y$'s. Instead, it decomposes the model into parts — nodes and edges — and computes the posteriors for those parts, that is, $P(Y_i=y_i\,|\,X=x)$ and $P(Y_i=y_i, Y_{i+1}=y_{i+1}\,|\,X=x)$. The crucial point is that these quantities can be computed using the forward-backward algorithm.

* Instead of updating the features for all possible outputs $y′ ∈ \Lambda^N$, we again exploit the decomposition into parts above and update only “local features” at the nodes and edges. [TODO clarify this]

### Pseudo Core

Below is pseudo code to optimize a CRF with the stochastic gradient descent (SGD) algorithm. Our toolkit also includes an implementation of a quasi-Newton method, L-BFGS, which converges faster. For the purpose of this exercise, however, we will stick with SGD.

[TODO pseudo code?]

## Exercises

Objectives:

* train a CRF using different feature sets for part-of-speech tagging;
* evaluate the model on the training, development and test sets.

Files used:

* class CRFOnline in lxmls/sequences/crf_online.py file
* class PostagCorpus in lxmls/sequences/readers/pos_corpus.py file
* class IDFeatures in lxmls/sequences/id_feature.py file
* class ExtendedFeatures in lxmls/sequences/extended_feature.py file

### Exercise 3.1 - Basic feature set

_Start by training the model. You will receive feedback when each epoch is finished. Note that running the 20 epochs might take a while._

In [ ]:
import lxmls.sequences.crf_online as crfo
import lxmls.readers.pos_corpus as pcc
import lxmls.sequences.id_feature as idfc
import lxmls.sequences.extended_feature as exfc

print "CRF Exercise"

# Load the corpus
corpus = pcc.PostagCorpus()

# Load the training, test and development sequences
train_seq = corpus.read_sequence_list_conll("data/train-02-21.conll", max_sent_len=10, max_nr_sent=1000)
test_seq = corpus.read_sequence_list_conll("data/test-23.conll", max_sent_len=10, max_nr_sent=1000)
dev_seq = corpus.read_sequence_list_conll("data/dev-22.conll", max_sent_len=10, max_nr_sent=1000)

# Build features
feature_mapper = idfc.IDFeatures(train_seq)
feature_mapper.build_features()

# Train the model
# You will receive feedback when each epoch is finished.
# Note that running the 20 epochs might take a while.
crf_online = crfo.CRFOnline(corpus.word_dict, corpus.tag_dict, feature_mapper)
crf_online.num_epochs = 20
crf_online.train_supervised(train_seq)

_Next, evaluate the model on the various datasets._

In [ ]:
# Make predictions for the various sequences using the trained model.
pred_train = crf_online.viterbi_decode_corpus(train_seq)
pred_dev = crf_online.viterbi_decode_corpus(dev_seq)
pred_test = crf_online.viterbi_decode_corpus(test_seq)

# Evaluate and print accuracies
eval_train = crf_online.evaluate_corpus(train_seq, pred_train)
eval_dev = crf_online.evaluate_corpus(dev_seq, pred_dev)
eval_test = crf_online.evaluate_corpus(test_seq, pred_test)
print "CRF - ID Features Accuracy Train: %.3f Dev: %.3f Test: %.3f"%(eval_train,eval_dev, eval_test)

_Your output should be similar to this:_

_CRF - ID Features Accuracy Train: 0.949 Dev: 0.846 Test: 0.858_

_Compare with the results achieved with the HMM model (0.837 on the test set). Even when using a similar feature set, a CRF yields better results than the HMM from the previous lecture._

_Perform some error analysis and figure out what are the main errors the tagger is making. Compare them with the errors made by the HMM model._

_**Hint:** use the methods developed in the previous lecture to help you with the error analysis._

### Exercise 3.2 - Extended feature set

_Train the model again, this time using the extended feature set._

In [ ]:
# Build features
feature_mapper = exfc.ExtendedFeatures(train_seq)
feature_mapper.build_features()

# Train the model
# You will receive feedback when each epoch is finished.
# Note that running the 20 epochs might take a while.
crf_online = crfo.CRFOnline(corpus.word_dict, corpus.tag_dict, feature_mapper)
crf_online.num_epochs = 20
crf_online.train_supervised(train_seq)

_And compute its accuracy._

In [ ]:
# Make predictions for the various sequences using the trained model.
pred_train = crf_online.viterbi_decode_corpus(train_seq)
pred_dev = crf_online.viterbi_decode_corpus(dev_seq)
pred_test = crf_online.viterbi_decode_corpus(test_seq)

# Evaluate and print accuracies
eval_train = crf_online.evaluate_corpus(train_seq, pred_train)
eval_dev = crf_online.evaluate_corpus(dev_seq, pred_dev)
eval_test = crf_online.evaluate_corpus(test_seq, pred_test)
print "CRF - ID Features Accuracy Train: %.3f Dev: %.3f Test: %.3f"%(eval_train,eval_dev, eval_test)

_Your output should be similar to this:_

_CRF - Extended Features Accuracy Train: 0.984 Dev: 0.899 Test: 0.894_

_Compare the errors obtained with the two different feature sets. Do some error analysis: what errors were correct by using more features? Can you think of other features to use to solve the errors you found?_

The main lesson from this exercise is that, if you are not satisfied by the accuracy of your algorithm, you can perform some error analysis and find out which errors your algorithm is making. You can then add more features which attempt to improve those specific errors — this is known as feature engineering.

Adding engineered features can lead to two problems:
* More features will make training and decoding more expensive. For example, if you add features that depend on the current word and the previous word, the number of new features is the square of the number of different words, which is quite large. For example, the Penn Treebank has around 40000 different words, so you are adding a lot of new features, even though not all pairs of words will ever occur. Features that depend on three words (previous, current, and next) are even more numerous.

* If features are very specific, such as the (previous word, current word, next word) one just mentioned, they might occur very rarely in the training set, which leads to overfit problems. Some of these problems (not all) can be mitigated with techniques such as smoothing, which you already learned about.


## Structured Perceptron

### Exercise 3.3 - Algorithm implementation

### Exercise 3.4 - POS Tagging